In [1]:
#Importing read-sm-files.py
import ReadSMFIles
from Resource import Resource
from Task import Task
from Schedule import Schedule
import random
from typing import List, Tuple

#reading j30.sm/j301_1.sm file
sm_file = ReadSMFIles.SMFileParser.parse_sm_file("j30.sm/j301_1.sm")

print(sm_file)


Resources:
   renewable  nonrenewable  doubly_constrained
0          4             0                   0

Project Information:
{'pronr': 1, 'jobs': 30, 'rel_date': 0, 'duedate': 38, 'tardcost': 26, 'MPM_Time': 38}

Precedence Relations:
    jobnr  modes  successors_count    successors
0       1      1                 3     [2, 3, 4]
1       2      1                 3   [6, 11, 15]
2       3      1                 3    [7, 8, 13]
3       4      1                 3    [5, 9, 10]
4       5      1                 1          [20]
5       6      1                 1          [30]
6       7      1                 1          [27]
7       8      1                 3  [12, 19, 27]
8       9      1                 1          [14]
9      10      1                 2      [16, 25]
10     11      1                 2      [20, 26]
11     12      1                 1          [14]
12     13      1                 2      [17, 18]
13     14      1                 1          [17]
14     15      1            

In [2]:
#Creating resources
r1 = int(sm_file[4].R1[0])
r2 = int(sm_file[4].R2[0])
r3 = int(sm_file[4].R3[0])
r4 = int(sm_file[4].R4[0])

R1 = Resource('R1', r1)
R2 = Resource('R2', r2)
R3 = Resource('R3', r3)
R4 = Resource('R4', r4)

resources = [R1, R2, R3, R4]

print([resource.name for resource in resources], [resource.per_period_availability for resource in resources])


['R1', 'R2', 'R3', 'R4'] [12, 13, 4, 12]


In [3]:
#Creating jobs
jobs_enumerate = sm_file[3].jobnr
jobs_duration = sm_file[3].duration
jobs_resources = sm_file[3].resources
jobs_successors = sm_file[2].successors

jobs = [None for _ in jobs_enumerate]


for i in jobs_enumerate:
    jobs[i - 1] = Task(str(i), jobs_duration[i - 1])

for i in range(len(resources)):
    for j in range(len(jobs)):
        jobs[j].add_renewable_resource(resources[i], jobs_resources[j][i])

for i in range(len(jobs)):
    successors = jobs_successors[i]
    for j in successors:
        jobs[i].add_sucessor(jobs[j - 1])
    
for job in jobs:
    print(job.name, [sucessor.name for sucessor in job.predecessors])

    

    



1 []
2 ['1']
3 ['1']
4 ['1']
5 ['4']
6 ['2']
7 ['3']
8 ['3']
9 ['4']
10 ['4']
11 ['2']
12 ['8']
13 ['3']
14 ['9', '12']
15 ['2']
16 ['10']
17 ['13', '14']
18 ['13']
19 ['8']
20 ['5', '11', '18']
21 ['16']
22 ['16', '17', '18']
23 ['20', '22']
24 ['19', '23']
25 ['10', '15', '20']
26 ['11']
27 ['7', '8']
28 ['21', '27']
29 ['19']
30 ['6', '24', '25']
31 ['26', '28']
32 ['29', '30', '31']



Genetic Algorithm



In [4]:
#Genetic Algorithm

#Creating random population
def create_population(population_size, jobs, resources: List[Resource]):
    population = []
    for i in range(population_size):
        schedule = Schedule()
        schedule.add_renewable_resources(resources)
        copy = jobs.copy()
        random.shuffle(copy)
        schedule.add_tasks(copy)
        population.append(schedule)
    
    return population

def create_population_with_valid_individuals(population_size: int, jobs: List[Task], resources: List[Resource]) -> List[Schedule]:
    population = []
    for _ in range(population_size):
        schedule = Schedule()
        schedule.add_renewable_resources(resources)
        _jobs = jobs.copy()
        while len(_jobs) > 0:
            job = random.choice(_jobs)
            schedule.add_task(job)
            if schedule.is_valid_precedence_relations_constraint():
                _jobs.remove(job)
            else:
                schedule.remove_task(job)
        population.append(schedule)
    return population


population = create_population(200, jobs, resources)

for individual in population:
    print([task.name for task in individual.tasks], individual.makespan())


['Dummy Source', '9', '30', '28', '21', '6', '27', '4', '12', '7', '29', '8', '1', '17', '3', '18', '16', '14', '23', '10', '15', '26', '2', '31', '19', '5', '20', '13', '22', '11', '25', '32', '24', 'Dummy Sink'] 46
['Dummy Source', '12', '11', '23', '32', '13', '28', '26', '14', '3', '2', '24', '7', '30', '16', '21', '19', '10', '8', '4', '18', '27', '22', '6', '29', '17', '9', '1', '15', '25', '5', '31', '20', 'Dummy Sink'] 46
['Dummy Source', '1', '3', '29', '16', '25', '7', '27', '12', '10', '15', '5', '26', '11', '32', '31', '2', '14', '23', '22', '20', '8', '21', '13', '6', '4', '18', '28', '17', '24', '19', '30', '9', 'Dummy Sink'] 51
['Dummy Source', '4', '1', '25', '22', '15', '29', '12', '30', '3', '16', '18', '10', '8', '6', '9', '5', '21', '28', '7', '31', '20', '19', '23', '17', '2', '11', '24', '27', '13', '14', '26', '32', 'Dummy Sink'] 53
['Dummy Source', '12', '18', '24', '22', '11', '8', '29', '17', '21', '27', '14', '30', '10', '19', '1', '31', '4', '5', '26', '3', 

Crossover

In [5]:

class OnePointCrossoverOperator:
    def __init__(self, crossover_rate: float = 0.8):
        self.crossover_rate = crossover_rate

    def __call__(
            self, parent1: Schedule, parent2: Schedule
    ) -> Tuple[Schedule, Schedule]:
        if random.random() > self.crossover_rate:
            return parent1, parent2

        n = len(parent1.tasks)
        crossover_point = random.randint(1, n - 3)

        child1 = Schedule()
        child2 = Schedule()

        parent1_tasks = parent1.tasks.copy()[1:-1]
        parent2_tasks = parent2.tasks.copy()[1:-1]

        child1.add_tasks(parent1_tasks[:crossover_point])
        for task in parent2.tasks:
            if task not in child1.tasks and task.name != 'Dummy Source' and task.name != 'Dummy Sink':
                child1.add_task(task)
        
        child2.add_tasks(parent2_tasks[:crossover_point])
        for task in parent1.tasks:
            if task not in child2.tasks and task.name != 'Dummy Source' and task.name != 'Dummy Sink':
                child2.add_task(task)

        return child1, child2


crossover = OnePointCrossoverOperator()

offspring = []
for i in range(0, len(population)):
    parent1 = random.choice(population)
    parent2 = random.choice(population)
    child1, child2 = crossover(parent1, parent2)
    offspring.append(child1)


#PRINTING TIME INTERVALS
# for task in child1.tasks:
#     print(task.earliest_start, task.earliest_finish, task.duration)




Parent Selection

In [6]:
class TournamentSelector:
    def __init__(self, tournament_size: int = 3):
        self.tournament_size = tournament_size

    def __call__(self, population: List[List[Task]]) -> List[Task]:
        selected = []
        for _ in range(len(population)):
            tournament = random.sample(population, self.tournament_size)
            winner = min(tournament, key=lambda x: x.makespan())
            selected.append(winner)
        return selected


selector = TournamentSelector(tournament_size=10)

selected = selector(population)

for i in range(len(selected)):
    print(selected[i].is_valid_precedence_relations_constraint(), [task.name for task in selected[i].tasks], selected[i].makespan())

False ['Dummy Source', '5', '27', '17', '31', '4', '12', '6', '8', '23', '25', '11', '13', '32', '15', '21', '20', '10', '30', '7', '1', '14', '29', '22', '26', '3', '2', '24', '19', '9', '16', '28', '18', 'Dummy Sink'] 54
False ['Dummy Source', '15', '5', '25', '2', '13', '11', '7', '23', '21', '30', '26', '16', '31', '8', '3', '24', '20', '14', '29', '28', '18', '10', '9', '4', '27', '22', '12', '6', '1', '32', '19', '17', 'Dummy Sink'] 55
False ['Dummy Source', '23', '24', '9', '30', '12', '3', '16', '25', '26', '11', '7', '17', '6', '15', '18', '20', '31', '22', '1', '28', '32', '29', '4', '8', '27', '10', '21', '19', '14', '5', '13', '2', 'Dummy Sink'] 62
False ['Dummy Source', '19', '5', '7', '3', '26', '20', '29', '1', '15', '21', '10', '27', '11', '30', '18', '23', '6', '14', '13', '8', '24', '17', '9', '28', '22', '12', '25', '16', '32', '31', '2', '4', 'Dummy Sink'] 47
False ['Dummy Source', '17', '2', '14', '32', '6', '7', '13', '30', '1', '26', '11', '16', '3', '20', '15', 

In [7]:
class SwapMutationOperator:
    def __init__(self, mutation_rate: float = 0.2):
        self.mutation_rate = mutation_rate

    def __call__(self, individual: Schedule) -> Schedule:
        if random.random() > self.mutation_rate:
            return individual

        n = len(individual.tasks)
        i, j = random.sample(range(1, n - 1), 2)
        print(i, j)
        individual.tasks[i], individual.tasks[j] = individual.tasks[j], individual.tasks[i]
        return individual

print([task.name for task in population[0].tasks], individual.makespan())
mutator = SwapMutationOperator()
population2 = population.copy()
individual = population2[0]
mutated_individual = mutator(individual)

print([task.name for task in mutated_individual.tasks], mutated_individual.makespan())

['Dummy Source', '9', '30', '28', '21', '6', '27', '4', '12', '7', '29', '8', '1', '17', '3', '18', '16', '14', '23', '10', '15', '26', '2', '31', '19', '5', '20', '13', '22', '11', '25', '32', '24', 'Dummy Sink'] 56
11 29
['Dummy Source', '9', '30', '28', '21', '6', '27', '4', '12', '7', '29', '11', '1', '17', '3', '18', '16', '14', '23', '10', '15', '26', '2', '31', '19', '5', '20', '13', '22', '8', '25', '32', '24', 'Dummy Sink'] 61
